### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup

### Load the dataset

In [2]:
sales = pd.read_csv('sales.csv')
sales.head()

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Load the dictionary created from API and convert to dataframe

In [3]:
games_dict_df = pd.DataFrame.from_dict(
    np.load('games_dict.npy', allow_pickle=True).item(), orient='index'
).reset_index(level=0).rename(columns={'index':'name'})
games_dict_df.head()

,name,release_date,metacritic_score
0,Wii Sports,2006-11-19,NaN
1,Super Mario Bros.,1985-09-13,NaN
2,Mario Kart Wii,2008-04-27,NaN
3,Wii Sports Resort,2009-07-26,NaN
4,Pokemon Red/Pokemon Blue,1996-02-27,NaN


### Join `sales` and `games_dict_df`

In [4]:
game_sales = pd.merge(sales, games_dict_df, how='left', left_on='name', right_on='name')
game_sales.head()

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales,release_date,metacritic_score
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,2006-11-19,NaN
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,1985-09-13,NaN
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,2008-04-27,NaN
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,2009-07-26,NaN
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,1996-02-27,NaN


### Explore the dataset and the target website

I noticed that a lot of games are missing their `metacritic_score` (they weren't uploaded to the database from where I pulled the information via API calls), so I'm going directly to the source, [Metacritic](http://metacritic.com). Similarly to the API exercise, I'm filtering my dataset to those games where `global_sales > 5` to limit the number of attempts.

In [5]:
tmp_games = game_sales[game_sales['global_sales']>5]
missing_metacritic_score = tmp_games[pd.isnull(tmp_games['metacritic_score'])].reset_index(drop=True)
missing_metacritic_score[['name', 'platform', 'global_sales', 'metacritic_score']].head()

,name,platform,global_sales,metacritic_score
0,Wii Sports,Wii,82.74,NaN
1,Super Mario Bros.,NES,40.24,NaN
2,Mario Kart Wii,Wii,35.82,NaN
3,Wii Sports Resort,Wii,33.00,NaN
4,Pokemon Red/Pokemon Blue,GB,31.37,NaN


The links on Metacritic are constructed in a way that requires the platform as well as the name of the game without special characters and with hyphens instead of spaces. I'm adding a column to the dataset with the constructed URL.

* Create a function to take care of mappings and URL construction

In [6]:
def construct_metacritic_url(platform, name):
    # Platforms are not abbreviated on Metacritic
    if platform == 'GBA':
        url_platform = 'game-boy-advance'
    elif platform == 'XB':
        url_platform = 'xbox'
    elif platform == 'X360':
        url_platform = 'xbox-360'
    elif platform == 'PS':
        url_platform = 'playstation'
    elif platform == 'PS2':
        url_platform = 'playstation-2'
    elif platform == 'PS3':
        url_platform = 'playstation-3'
    elif platform == 'PS4':
        url_platform = 'playstation-4'
    elif platform == 'N64':
        url_platform = 'nintendo-64'
    elif platform == 'GC':
        url_platform = 'gamecube'
    elif platform == 'WiiU':
        url_platform = 'wii-u'
    else:
        url_platform = platform
        
    # Names don't have special characters and use hyphens instead of spaces
    url_name = name.replace(' ', '-').replace('/', '-').replace('.', '')
        
    return('http://metacritic.com/game/'+url_platform.lower()+'/'+url_name.lower())

In [7]:
missing_metacritic_score['metacritic_url'] = missing_metacritic_score.apply(
    lambda x: construct_metacritic_url(x['platform'], x['name']), axis=1
)
missing_metacritic_score[['name', 'platform', 'metacritic_url']].head()

,name,platform,metacritic_url
0,Wii Sports,Wii,http://metacritic.com/game/wii/wii-sports
1,Super Mario Bros.,NES,http://metacritic.com/game/nes/super-mario-bros
2,Mario Kart Wii,Wii,http://metacritic.com/game/wii/mario-kart-wii
3,Wii Sports Resort,Wii,http://metacritic.com/game/wii/wii-sports-resort
4,Pokemon Red/Pokemon Blue,GB,http://metacritic.com/game/gb/pokemon-red-poke...


* Explore the website with a specific example

In [8]:
res = requests.get('http://metacritic.com/game/wii/wii-sports', headers={'User-agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
   "https://www.w3.org/TR/html4/strict.dtd">
<html xml:lang="en">
 <head>
  <title>
   Wii Sports for Wii Reviews - Metacritic
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).loader_config={licenseKey:"860f7644b8",applicationID:"2033002"};window.NREUM||(NREUM={}),__nr_require=function(n,e,t){function r(t){if(!e[t]){var i=e[t]={exports:{}};n[t][0].call(i.exports,function(e){var i=n[t][1][e];return r(i||e)},i,i.exports)}return e[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(n,e,t){function r(){}function i(n,e,t){return function(){return o(n,[u.now()].concat(f(arguments)),e?null:this,t),e?void 0:this}}var o=n("handle"),a=n(4),f=n(5),c=n("ee").get("tracer"),u=n("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttri

The metascore on Metacritic is in a div class called `metascore_w`.

In [9]:
soup.find_all('div', class_='metascore_w')

[<div class="metascore_w medium movie positive">67</div>,
 <div class="metascore_w medium movie mixed">50</div>,
 <div class="metascore_w medium movie mixed">44</div>,
 <div class="metascore_w medium movie negative">37</div>,
 <div class="metascore_w medium game positive">89</div>,
 <div class="metascore_w medium game positive">88</div>,
 <div class="metascore_w medium game positive">86</div>,
 <div class="metascore_w medium game positive">83</div>,
 <div class="metascore_w medium game mixed">73</div>,
 <div class="metascore_w medium season positive">78</div>,
 <div class="metascore_w medium season positive">72</div>,
 <div class="metascore_w medium season positive">71</div>,
 <div class="metascore_w medium season positive">70</div>,
 <div class="metascore_w medium season positive">61</div>,
 <div class="metascore_w medium release positive">92</div>,
 <div class="metascore_w medium release positive">89</div>,
 <div class="metascore_w medium release positive">85</div>,
 <div class="meta

But that returns all scores from the reviews as well, however, it is the only score that uses the `xlarge` class, so I can extract that.

In [10]:
soup.find_all('div', class_='xlarge')[0].text

'76'

### Create a loop to pull all available scores from Metacritic to a dictionary

* Utilizing `time.sleep` with a random number between 1 and 1.5 seconds to avoid ban for scraping
* The `meta_key` that I use for the dictionary's key has to be a combination of the `name` and the `platform` from the original dataset, because one game can show up on the list for multiple platforms
* Not all games are available on Metacritic (especially older ones), so I need to do some error-handling for 404 response
* There are games that haven't been reviewed yet, so I have to see if `xlarge` returns anything first, before pulling the text

In [11]:
header = {'User-agent': 'Mozilla/5.0'}
metacritic_dict = {}

for i in range(len(missing_metacritic_score)):
    meta_key = missing_metacritic_score['name'][i]+'_'+missing_metacritic_score['platform'][i]
    
    time.sleep(np.random.uniform(1, 1.5))
    r = requests.get(missing_metacritic_score['metacritic_url'][i], headers=header)
    
    if r.status_code == 404:
        continue
    else:  
        xlarge_div = BeautifulSoup(r.text, 'html.parser').find_all('div', class_='xlarge')
        if len(xlarge_div) > 0:
            meta_score = xlarge_div[0].text  
        else:
            meta_score = None
        
        metacritic_dict.update({meta_key: meta_score})

In [12]:
metacritic_dict

{'Wii Sports_Wii': '76',
 'Mario Kart Wii_Wii': '82',
 'Wii Sports Resort_Wii': '80',
 'New Super Mario Bros._DS': '89',
 'Wii Play_Wii': '58',
 'New Super Mario Bros. Wii_Wii': '87',
 'Mario Kart DS_DS': '91',
 'Wii Fit_Wii': '80',
 'Wii Fit Plus_Wii': '80',
 'Kinect Adventures!_X360': '61',
 'Super Smash Bros. Brawl_Wii': '93',
 'Mario Kart 7_3DS': '85',
 'Halo 3_X360': '94',
 'Super Mario 64_N64': '94',
 'Super Mario Galaxy_Wii': '97',
 'Gran Turismo_PS': '96',
 'Super Mario 3D Land_3DS': '90',
 'Gran Turismo 5_PS3': '84',
 'Just Dance 3_Wii': '74',
 'Mario Kart 64_N64': '83',
 'New Super Mario Bros. 2_3DS': '78',
 'Halo 4_X360': '87',
 'Final Fantasy VII_PS': '92',
 'Just Dance 2_Wii': '74',
 'Gran Turismo 2_PS': '93',
 'Minecraft_X360': '82',
 'Mario Party DS_DS': '72',
 'FIFA 16_PS4': '82',
 'Wii Party_Wii': '68',
 'Halo 2_XB': '95',
 'Mario Party 8_Wii': '62',
 'FIFA Soccer 13_PS3': '88',
 'GoldenEye 007_N64': '96',
 'Final Fantasy X_PS2': '92',
 'Final Fantasy VIII_PS': '90',
 

### Save the dictionary

In [13]:
np.save('metacritic_dict.npy', metacritic_dict)